For this experiment, we would try to attempt to find candlestick patterns, and pattern combinations that are profitable. We would employ a genetic algorithm to select the best patterns over a number of generations. This follows the instruction from neurotrader on Youtube

The steps for the genetic algorithm are as follows:
1. Initialization : Generation of random patterns to make up the starting pattern population. 
2. Evaluation : Over the same data, each patterns are evaluated for a performance metric. Fitness Values are then assigned to each pattern in the population.
3. Elitism : The pattern population is then sorted by their fitness values, and the best (single pattern or percentage of population) are selected for the next generation. 
4. Parent Selection : Selection of two parents to form children patterns for the next generation.
5. Reproduction
6. Mutation

Parameters include:
- Pattern Size : How many comparison rules/conditions would make up one pattern
- Number of lookback candle : How many historical candles to consider
- Performance metric : Martin Ratio (default), Profit Factor, Total Return
- Stop Criteria
- Population Size
- Number of Generations
- Elitism Selection Count
- Mutation Chance
- Fresh Pattern Chance
- Minimum number of patterns

In [28]:
from enum import Enum, auto
from typing import Tuple, Union


class DataSource(Enum):
    OPEN = auto()
    HIGH = auto()
    LOW = auto()
    CLOSE = auto()

    # Aliases
    O = OPEN #noqa
    H = HIGH
    L = LOW
    C = CLOSE


class Operator(Enum):
    GREATER_THAN = ">"
    LESS_THAN = "<"
    GREATER_OR_EQUAL = ">="
    LESS_OR_EQUAL = "<="
    EQUAL = "=="
    NOT_EQUAL = "!="

    def compare(self, *values):
        """Compares values using the enum member's operator.

        Args:
            *values: Either two values (value_1, value_2) or a tuple of two values.

        Returns:
            bool: True if the comparison holds, False otherwise.
        """
        if len(values) == 2:
            value_1, value_2 = values
        elif len(values) == 1:
            value_1, value_2 = values[0]
        else:
            raise ValueError("Invalid number of values for comparison.")

        if self == Operator.GREATER_THAN:
            return value_1 > value_2
        elif self == Operator.LESS_THAN:
            return value_1 < value_2
        elif self == Operator.GREATER_OR_EQUAL:
            return value_1 >= value_2
        elif self == Operator.LESS_OR_EQUAL:
            return value_1 <= value_2
        elif self == Operator.EQUAL:
            return value_1 == value_2
        elif self == Operator.NOT_EQUAL:
            return value_1 != value_2


class PatternComponent:
    """
    This class represents a component of a pattern used for comparison.
    """
    def __init__(self, source_pair: Tuple[DataSource, DataSource], index_pair:Tuple[int, int], operator: Operator) -> None:
        self.source_pair = source_pair
        self.index_pair = index_pair
        self.operator = operator

        self.value : Tuple[Union[int, float], Union[int, float]] = (None, None)